In [4]:
!pip install iterative-stratification

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
  Using cached iterative_stratification-0.1.6-py3-none-any.whl (8.7 kB)


In [1]:
import pandas as pd
import numpy as np
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

In [2]:
train_labels = pd.read_csv("/home/ubuntu/datasets/human-protein-atlas/train.csv")
train_labels.head()

,Id,Target
0,00070df0-bbc3-11e8-b2bc-ac1f6b6435d0,16 0
1,000a6c98-bb9b-11e8-b2b9-ac1f6b6435d0,7 1 2 0
2,000a9596-bbc4-11e8-b2bc-ac1f6b6435d0,5
3,000c99ba-bba4-11e8-b2b9-ac1f6b6435d0,1
4,001838f8-bbca-11e8-b2bc-ac1f6b6435d0,18


In [40]:
# from sklearn.preprocessing import MultiLabelBinarizer

# one_hot = MultiLabelBinarizer()

# # One-hot encode data
# y = one_hot.fit_transform(df["Target"])

In [3]:
label_names = {
    0:  "Nucleoplasmn",  
    1:  "Nuclear membrane",   
    2:  "Nucleoli",   
    3:  "Nucleoli fibrillar center",   
    4:  "Nuclear speckles",
    5:  "Nuclear bodies",   
    6:  "Endoplasmic reticulum",   
    7:  "Golgi apparatus",   
    8:  "Peroxisomes",   
    9:  "Endosomes",   
    10:  "Lysosomes", 
    11:  "Intermediate filaments",   
    12:  "Actin filaments",   
    13:  "Focal adhesion sites",   
    14:  "Microtubules",   
    15:  "Microtubule ends",   
    16:  "Cytokinetic bridge",   
    17:  "Mitotic spindle",   
    18:  "Microtubule organizing center",   
    19:  "Centrosome",   
    20:  "Lipid droplets",   
    21:  "Plasma membrane",   
    22:  "Cell junctions",   
    23:  "Mitochondria",   
    24:  "Aggresome",   
    25:  "Cytosol",   
    26:  "Cytoplasmic bodies",   
    27:  "Rods & rings"
}

In [4]:
reverse_train_labels = dict((v,k) for k,v in label_names.items())

def fill_targets(row):
    row.Target = np.array(row.Target.split(" ")).astype(np.int)
    for num in row.Target:
        name = label_names[int(num)]
        row.loc[name] = 1
    return row

In [5]:
for key in label_names.keys():
    train_labels[label_names[key]] = 0

train_labels = train_labels.apply(fill_targets, axis=1)

In [6]:
train_labels.head()

,Id,Target,Nucleoplasmn,Nuclear membrane,Nucleoli,Nucleoli fibrillar center,Nuclear speckles,Nuclear bodies,Endoplasmic reticulum,Golgi apparatus,...,Microtubule organizing center,Centrosome,Lipid droplets,Plasma membrane,Cell junctions,Mitochondria,Aggresome,Cytosol,Cytoplasmic bodies,Rods & rings
0,00070df0-bbc3-11e8-b2bc-ac1f6b6435d0,"[16, 0]",1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,000a6c98-bb9b-11e8-b2b9-ac1f6b6435d0,"[7, 1, 2, 0]",1,1,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,000a9596-bbc4-11e8-b2bc-ac1f6b6435d0,[5],0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,000c99ba-bba4-11e8-b2b9-ac1f6b6435d0,[1],0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,001838f8-bbca-11e8-b2bc-ac1f6b6435d0,[18],0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [7]:
train_labels.shape

(31072, 30)

In [8]:
train_labels.loc[:,"kfold"] = -1

In [9]:
train_labels.columns

Index(['Id', 'Target', 'Nucleoplasmn', 'Nuclear membrane', 'Nucleoli',
       'Nucleoli fibrillar center', 'Nuclear speckles', 'Nuclear bodies',
       'Endoplasmic reticulum', 'Golgi apparatus', 'Peroxisomes', 'Endosomes',
       'Lysosomes', 'Intermediate filaments', 'Actin filaments',
       'Focal adhesion sites', 'Microtubules', 'Microtubule ends',
       'Cytokinetic bridge', 'Mitotic spindle',
       'Microtubule organizing center', 'Centrosome', 'Lipid droplets',
       'Plasma membrane', 'Cell junctions', 'Mitochondria', 'Aggresome',
       'Cytosol', 'Cytoplasmic bodies', 'Rods & rings', 'kfold'],
      dtype='object')

In [10]:
# random shuffle with fraction = 1 then reset the index
train_labels = train_labels.sample(frac=1).reset_index(drop=True)

X = train_labels.Id.values
#y = df[["grapheme_root","vowel_diacritic","consonant_diacritic"]].values
y = train_labels.iloc[:,2:].values

# define how many folds
mskf = MultilabelStratifiedKFold(n_splits=5)

In [11]:
X[:5], y[:5]

(array(['7740d502-bbc0-11e8-b2bb-ac1f6b6435d0',
        '086354a6-bbad-11e8-b2ba-ac1f6b6435d0',
        '7b839ed4-bbb0-11e8-b2ba-ac1f6b6435d0',
        'dd15a0a4-bbc7-11e8-b2bc-ac1f6b6435d0',
        '9343c5dc-bb9f-11e8-b2b9-ac1f6b6435d0'], dtype=object),
 array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0, -1],
        [ 1,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1],
        [ 1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0, -1],
        [ 0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1],
        [ 1,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0, -1]]))

In [13]:
# Let's go through the iterator and get train and val indices
for fold, (trn_, val_) in enumerate(mskf.split(X, y)):
    print("TRAIN: ", trn_, "VALID : ",val_)
    train_labels.loc[val_, "kfold"] = fold

TRAIN:  [    0     1     2 ... 31068 31070 31071] VALID :  [    4     5    12 ... 31059 31062 31069]
TRAIN:  [    1     2     3 ... 31069 31070 31071] VALID :  [    0     9    11 ... 31049 31051 31068]
TRAIN:  [    0     1     3 ... 31069 31070 31071] VALID :  [    2     8    15 ... 31055 31064 31066]
TRAIN:  [    0     2     3 ... 31069 31070 31071] VALID :  [    1     6     7 ... 31050 31056 31067]
TRAIN:  [    0     1     2 ... 31067 31068 31069] VALID :  [    3    10    13 ... 31065 31070 31071]


In [16]:
train_labels["kfold"]

0        1
1        3
2        2
3        4
4        0
        ..
31067    3
31068    1
31069    0
31070    4
31071    4
Name: kfold, Length: 31072, dtype: int64

In [17]:
train_labels["kfold"].value_counts()

4    6215
3    6215
2    6214
1    6214
0    6214
Name: kfold, dtype: int64

Let's check a few values to ensure the splitting went as expected

In [23]:
# Common category
train_labels.groupby('kfold')['Nucleoplasmn'].sum()

kfold
0    2629
1    2550
2    2581
3    2552
4    2573
Name: Nucleoplasmn, dtype: int64

In [24]:
train_labels.groupby('kfold')['Nucleoli'].sum()

kfold
0    725
1    724
2    724
3    724
4    724
Name: Nucleoli, dtype: int64

In [25]:
# Rare category
train_labels.groupby('kfold')['Peroxisomes'].sum()

kfold
0    10
1    11
2    11
3    11
4    10
Name: Peroxisomes, dtype: int64

In [26]:
# Rare category
train_labels.groupby('kfold')['Aggresome'].sum()

kfold
0    64
1    65
2    65
3    64
4    64
Name: Aggresome, dtype: int64

__^ Looks like the classes got equally distributed in all folds as expected.__

In [28]:
train_labels.to_csv("../input/train_folds.csv", index=False)

In [29]:
# if __name__ == "__main__":
    
#     label_names = {
#     0:  "Nucleoplasmn",  
#     1:  "Nuclear membrane",   
#     2:  "Nucleoli",   
#     3:  "Nucleoli fibrillar center",   
#     4:  "Nuclear speckles",
#     5:  "Nuclear bodies",   
#     6:  "Endoplasmic reticulum",   
#     7:  "Golgi apparatus",   
#     8:  "Peroxisomes",   
#     9:  "Endosomes",   
#     10:  "Lysosomes", 
#     11:  "Intermediate filaments",   
#     12:  "Actin filaments",   
#     13:  "Focal adhesion sites",   
#     14:  "Microtubules",   
#     15:  "Microtubule ends",   
#     16:  "Cytokinetic bridge",   
#     17:  "Mitotic spindle",   
#     18:  "Microtubule organizing center",   
#     19:  "Centrosome",   
#     20:  "Lipid droplets",   
#     21:  "Plasma membrane",   
#     22:  "Cell junctions",   
#     23:  "Mitochondria",   
#     24:  "Aggresome",   
#     25:  "Cytosol",   
#     26:  "Cytoplasmic bodies",   
#     27:  "Rods & rings"
#     }
    
#     train_labels = pd.read_csv("/home/ubuntu/datasets/human-protein-atlas/train.csv")
#     print(train_labels.head())
    
#     reverse_train_labels = dict((v,k) for k,v in label_names.items())

#     def fill_targets(row):
#         row.Target = np.array(row.Target.split(" ")).astype(np.int)
#         for num in row.Target:
#             name = label_names[int(num)]
#             row.loc[name] = 1
#         return row
    
#     for key in label_names.keys():
#         train_labels[label_names[key]] = 0
        
#     train_labels = train_labels.apply(fill_targets, axis=1)
    
    
#     train_labels.loc[:,"kfold"] = -1
    
#     # random shuffle with fraction = 1 then reset the index
#     train_labels = train_labels.sample(frac=1).reset_index(drop=True)
    
#     X = train_labels.Id.values
#     #y = df[["grapheme_root","vowel_diacritic","consonant_diacritic"]].values
#     y = train_labels[["Target"]].values
    
#     # define how many folds
#     mskf = MultilabelStratifiedKFold(n_splits=5)
    
#     # Let's go through the iterator and get trai and al indices
#     for fold, (trn_, val_) in enumerate(mskf.split(X, y)):
#         print("TRAIN: ", trn_, "VALID : ",val_)
#         train_labels.loc[val_, "kfold"] = fold
        
#     print(df.kfold.value_counts())
#     train_labels.to_csv("../input/train_folds.csv", index=False)

In [ ]:
self.image_ids = df.Id.values
self.nucleoplasmn  = df["Nucleoplasmn"].values
self.nuclear_membrane  = df["Nuclear membrane"].values
self.nucleoli  = df["Nucleoli"].values
self.nucleoli_fibrillar_center  = df["Nucleoli fibrillar center"].values
self.nuclear_speckles  = df["Nuclear speckles"].values
self.nuclear_bodies  = df["Nuclear bodies"].values
self.endoplasmic_reticulum  = df["Endoplasmic reticulum"].values
self.golgi_apparatus  = df["Golgi apparatus"].values
self.peroxisomes  = df["Peroxisomes"].values
self.endosomes  = df["Endosomes"].values
self.lysosomes  = df["Lysosomes"].values
self.intermediate_filaments  = df["Intermediate filaments"].values
self.actin_filaments  = df["Actin filaments"].values
self.focal_adhesion_sites  = df["Focal adhesion sites"].values
self.microtubules  = df["Microtubules"].values
self.microtubule_ends  = df["Microtubule ends"].values
self.cytokinetic_bridge  = df["Cytokinetic bridge"].values
self.mitotic_spindle  = df["Mitotic spindle"].values
self.microtubule_organizing_center  = df["Microtubule organizing center"].values
self.centrosome  = df["Centrosome"].values
self.lipid_droplets = df["Lipid droplets"].values
self.plasma_membrane  = df["Plasma membrane"].values
self.cell_junctions  = df["Cell junctions"].values
self.mitochondria  = df["Mitochondria"].values
self.aggresome  = df["Aggresome"].values
self.cytosol  = df["Cytosol"].values
self.cytoplasmic_bodies  = df["Cytoplasmic bodies"].values
self.rods_rings  = df["Rods & rings"].values

In [ ]:
self.image_ids 
self.nucleoplasmn  
self.nuclear_membrane  
self.nucleoli  
self.nucleoli_fibrillar_center  
self.nuclear_speckles  
self.nuclear_bodies  
self.endoplasmic_reticulum  
self.golgi_apparatus  
self.peroxisomes  
self.endosomes  
self.lysosomes  
self.intermediate_filaments  
self.actin_filaments  
self.focal_adhesion_sites  
self.microtubules  
self.microtubule_ends  
self.cytokinetic_bridge  
self.mitotic_spindle  
self.microtubule_organizing_center  
self.centrosome  
self.lipid_droplets 
self.plasma_membrane  
self.cell_junctions  
self.mitochondria  
self.aggresome  
self.cytosol  
self.cytoplasmic_bodies  
self.rods_rings

In [ ]:
return{
    'image': torch.tensor(image, dtype=torch.float),
    'nucleoplasmn': torch.tensor(self.nucleoplasmn  , dtype=torch.long),
    'nuclear_membrane': torch.tensor(self.nuclear_membrane  , dtype=torch.long),
    'nucleoli': torch.tensor(self.nucleoli  , dtype=torch.long),
    'nucleoli_fibrillar_center': torch.tensor(self.nucleoli_fibrillar_center  , dtype=torch.long),
    'nuclear_speckles': torch.tensor(self.nuclear_speckles  , dtype=torch.long),
    'nuclear_bodies': torch.tensor(self.nuclear_bodies  , dtype=torch.long),
    'endoplasmic_reticulum': torch.tensor(self.endoplasmic_reticulum  , dtype=torch.long),
    'golgi_apparatus': torch.tensor(self.golgi_apparatus  , dtype=torch.long),
    'peroxisomes': torch.tensor(self.peroxisomes  , dtype=torch.long),
    'endosomes': torch.tensor(self.endosomes  , dtype=torch.long),
    'lysosomes': torch.tensor(self.lysosomes  , dtype=torch.long),
    'intermediate_filaments': torch.tensor(self.intermediate_filaments  , dtype=torch.long),
    'actin_filaments': torch.tensor(self.actin_filaments  , dtype=torch.long),
    'focal_adhesion_sites': torch.tensor(self.focal_adhesion_sites  , dtype=torch.long),
    'microtubules': torch.tensor(self.microtubules  , dtype=torch.long),
    'microtubule_ends': torch.tensor(self.microtubule_ends  , dtype=torch.long),
    'cytokinetic_bridge': torch.tensor(self.cytokinetic_bridge  , dtype=torch.long),
    'mitotic_spindle': torch.tensor(self.mitotic_spindle  , dtype=torch.long),
    'microtubule_organizing_center': torch.tensor(self.microtubule_organizing_center  , dtype=torch.long),
    'centrosome': torch.tensor(self.centrosome  , dtype=torch.long),
    'lipid_droplets': torch.tensor(self.lipid_droplets , dtype=torch.long),
    'plasma_membrane': torch.tensor(self.plasma_membrane  , dtype=torch.long),
    'cell_junctions': torch.tensor(self.cell_junctions  , dtype=torch.long),
    'mitochondria': torch.tensor(self.mitochondria  , dtype=torch.long),
    'aggresome': torch.tensor(self.aggresome  , dtype=torch.long),
    'cytosol': torch.tensor(self.cytosol  , dtype=torch.long),
    'cytoplasmic_bodies': torch.tensor(self.cytoplasmic_bodies  , dtype=torch.long),
    'rods_rings': torch.tensor(self.rods_rings, dtype=torch.long)
}

In [ ]:
self.l0 = nn.Linear(512, 2)  # nucleoplasmn
self.l1 = nn.Linear(512, 2)  # nuclear_membrane
self.l2 = nn.Linear(512, 2)  # nucleoli
self.l3 = nn.Linear(512, 2)  # nucleoli_fibrillar_center
self.l4 = nn.Linear(512, 2)  # nuclear_speckles
self.l5 = nn.Linear(512, 2)  # nuclear_bodies
self.l6 = nn.Linear(512, 2)  # endoplasmic_reticulum
self.l7 = nn.Linear(512, 2)  # golgi_apparatus
self.l8 = nn.Linear(512, 2)  # peroxisomes
self.l9 = nn.Linear(512, 2)  # endosomes
self.l10 = nn.Linear(512, 2)  # lysosomes
self.l11 = nn.Linear(512, 2)  # intermediate_filaments
self.l12 = nn.Linear(512, 2)  # actin_filaments
self.l13 = nn.Linear(512, 2)  # focal_adhesion_sites
self.l14 = nn.Linear(512, 2)  # microtubules
self.l15 = nn.Linear(512, 2)  # microtubule_ends
self.l16 = nn.Linear(512, 2)  # cytokinetic_bridge
self.l17 = nn.Linear(512, 2)  # mitotic_spindle
self.l18 = nn.Linear(512, 2)  # microtubule_organizing_center
self.l19 = nn.Linear(512, 2)  # centrosome
self.l20 = nn.Linear(512, 2)  # lipid_droplets
self.l21 = nn.Linear(512, 2)  # plasma_membrane
self.l22 = nn.Linear(512, 2)  # cell_junctions
self.l23 = nn.Linear(512, 2)  # mitochondria
self.l24 = nn.Linear(512, 2)  # aggresome
self.l25 = nn.Linear(512, 2)  # cytosol
self.l26 = nn.Linear(512, 2)  # cytoplasmic_bodies
self.l27 = nn.Linear(512, 2)  # rods_rings